In [54]:
import requests
import sqlite3
from datetime import datetime
import pandas as pd

In [55]:
df = pd.read_csv('meteo_paris_2019-2022.csv')
df = df.drop(columns=['elevation','utc_offset_seconds','timezone','timezone_abbreviation'])
df = df.rename(columns={'latitude':'date', 'longitude':'temperature'})
df = df.drop([0,1])
df['date'] = df['date'].apply(lambda x : x[8:10]+'/'+x[5:7]+'/'+x[0:4])
df = df[['temperature', 'date']]
df

,temperature,date
2,7.5,01/01/2019
3,5.7,02/01/2019
4,2.4,03/01/2019
5,1.5,04/01/2019
6,2.8,05/01/2019
...,...,...
1458,5.5,27/12/2022
1459,10.0,28/12/2022
1460,9.8,29/12/2022
1461,9.6,30/12/2022


In [56]:
conn = sqlite3.connect('bdd.db')

df.to_sql('temperature', conn, if_exists='append', index=False)

conn.close()

In [ ]:
def load_data():
    conn = sqlite3.connect("bdd.db")
    query = """
    SELECT objet.date, objet.gare, objet.type, objet.nature, gare.lat, gare.lon, temperature.temperature
    FROM objet
    JOIN gare ON objet.gare = gare.gare
    JOIN temperature ON objet.date = temperature.date
    """
    df = pd.read_sql_query(query, conn)
    conn.close()
    return df

df = load_data()
df['nombre_objets'] = 1

In [ ]:
def get_season(date_str):
    year = int(date_str[-4:])
    date = pd.to_datetime(date_str, format='%d/%m/%Y')
    month = date.month
    day = date.day
    
    if month == 12 and day == 31:
        return 'hiver'
    
    season_boundaries = [
        {'start': pd.to_datetime(f'20/12/{year-1}', format='%d/%m/%Y'), 'end': pd.to_datetime(f'20/03/{year}', format='%d/%m/%Y'), 'season': 'hiver'},
        {'start': pd.to_datetime(f'21/03/{year}', format='%d/%m/%Y'), 'end': pd.to_datetime(f'20/06/{year}', format='%d/%m/%Y'), 'season': 'printemps'},
        {'start': pd.to_datetime(f'21/06/{year}', format='%d/%m/%Y'), 'end': pd.to_datetime(f'20/09/{year}', format='%d/%m/%Y'), 'season': 'été'},
        {'start': pd.to_datetime(f'21/09/{year}', format='%d/%m/%Y'), 'end': pd.to_datetime(f'20/12/{year}', format='%d/%m/%Y'), 'season': 'automne'}
    ]
    
    for boundary in season_boundaries:
        if boundary['start'] <= date <= boundary['end']:
            return boundary['season']


df['saison'] = df['date'].apply(get_season)
df.to_csv('liste_objets_avec_saison.csv')